In [1]:
import pandas as pd
import numpy as np

from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV

from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import Perceptron
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier

from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import recall_score
from sklearn.metrics import precision_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import log_loss

import warnings
# warnings.filterwarnings('ignore')

### Clean Chess Dataset

In [2]:
# https://www.kaggle.com/datasnaek/chess
chess_df = pd.read_csv('Data/games.csv')
chess_df.head()

id  rated    created_at  last_move_at  turns victory_status winner  \
0  TZJHLljE  False  1.504210e+12  1.504210e+12     13      outoftime  white   
1  l1NXvwaE   True  1.504130e+12  1.504130e+12     16         resign  black   
2  mIICvQHh   True  1.504130e+12  1.504130e+12     61           mate  white   
3  kWKvrqYL   True  1.504110e+12  1.504110e+12     61           mate  white   
4  9tXo1AUZ   True  1.504030e+12  1.504030e+12     95           mate  white   

  increment_code       white_id  white_rating      black_id  black_rating  \
0           15+2       bourgris          1500          a-00          1191   
1           5+10           a-00          1322     skinnerua          1261   
2           5+10         ischia          1496          a-00          1500   
3           20+0  daniamurashov          1439  adivanov2009          1454   
4           30+3      nik221107          1523  adivanov2009          1469   

                                               moves opening_eco  \
0  d4 d5 c4 c6 cxd5 e6 dxe6 fxe6 Nf3 Bb4+ Nc3 Ba5...         D10   
1  d4 Nc6 e4 e5 f4 f6 dxe5 fxe5 fxe5 Nxe5 Qd4 Nc6...         B00   
2  e4 e5 d3 d6 Be3 c6 Be2 b5 Nd2 a5 a4 c5 axb5 Nc...         C20   
3  d4 d5 Nf3 Bf5 Nc3 Nf6 Bf4 Ng4 e3 Nc6 Be2 Qd7 O...         D02   
4  e4 e5 Nf3 d6 d4 Nc6 d5 Nb4 a3 Na6 Nc3 Be7 b4 N...         C41   

                             opening_name  opening_ply  
0        Slav Defense: Exchange Variation            5  
1  Nimzowitsch Defense: Kennedy Variation            4  
2   King's Pawn Game: Leonardis Variation            3  
3  Queen's Pawn Game: Zukertort Variation            3  
4                        Philidor Defense            5

In [3]:
chess_df['winner_white'] = chess_df['winner'] == 'white'
# chess_df = chess_df[['rated', 'created_at', 'last_move_at', 'turns', 'victory_status',
#                      'white_rating', 'black_rating', 'opening_eco', 'opening_ply', 'winner_white']]
chess_df = chess_df[['rated', 'turns', 'victory_status',
                     'white_rating', 'black_rating', 'opening_eco', 'opening_ply', 'winner_white']]
chess_df.head()

rated  turns victory_status  white_rating  black_rating opening_eco  \
0  False     13      outoftime          1500          1191         D10   
1   True     16         resign          1322          1261         B00   
2   True     61           mate          1496          1500         C20   
3   True     61           mate          1439          1454         D02   
4   True     95           mate          1523          1469         C41   

   opening_ply  winner_white  
0            5          True  
1            4         False  
2            3          True  
3            3          True  
4            5          True

In [4]:
chess_df_X = chess_df.drop(columns=['winner_white'])
chess_df_y = chess_df['winner_white']

In [5]:
chess_X_cat_col = ['rated', 'victory_status', 'opening_eco']
chess_X = pd.get_dummies(columns=chess_X_cat_col, data=chess_df_X)

chess_y = chess_df_y.replace({True: 1, False: 0})

### Clean Mushrooms Dataset

class: edible(e), poisonous(p)

cap-shape: bell(b), conical(c), convex(x), flat(f), knobbed(k), sunken(s)

cap-surface: fibrous(f), grooves(g), scaly(y), smooth(s)

cap-color: brown(n), buff(b), cinnamon(c), gray(g), green(r), pink(p), purple(u), red(e), white(w), yellow(y)

bruises: bruises(t), no(f)

odor: almond(a), anise(l), creosote(c), fishy(y), foul(f), musty(m), none(n), pungent(p), spicy(s)

gill-attachment: attached(a), descending(d), free(f), notched(n)

gill-spacing: close(c), crowded(w), distant(d)

gill-size: broad(b), narrow(n)

gill-color: black(k), brown(n), buff(b), chocolate(h), gray(g), green(r), orange(o), pink(p), purple(u), red(e), white(w), yellow(y)

stalk-shape: enlarging(e), tapering(t)

stalk-root: bulbous(b), club(c), cup(u), equal(e), rhizomorphs(z), rooted(r), missing(?)

stalk-surface-above-ring: fibrous(f), scaly(y), silky(k), smooth(s)

stalk-surface-below-ring: fibrous(f), scaly(y), silky(k), smooth(s)

stalk-color-above-ring: brown(n), buff(b), cinnamon(c), gray(g), orange(o), pink(p), red(e), white(w), yellow(y)

stalk-color-below-ring: brown(n), buff(b), cinnamon(c), gray(g), orange(o), pink(p), red(e), white(w), yellow(y)

veil-type: partial(p), universal(u)

veil-color: brown(n), orange(o), white(w), yellow(y)

ring-number: none(n), one(o), two(t)

ring-type: cobwebby(c), evanescent(e), flaring(f), large(l), none(n), pendant(p), sheathing(s), zone(z)

spore-print-color: black(k), brown(n), buff(b), chocolate(h), green(r), orange(o), purple(u), white(w), yellow(y)

population: abundant(a), clustered(c), numerous(n), scattered(s), several(v), solitary(y)

habitat: grasses(g), leaves(l), meadows(m), paths(p), urban(u), waste(w), woods(d)

In [6]:
# https://www.kaggle.com/uciml/mushroom-classification
shrooms = pd.read_csv('Data/mushrooms.csv')
shrooms.head()

class cap-shape cap-surface cap-color bruises odor gill-attachment  \
0     p         x           s         n       t    p               f   
1     e         x           s         y       t    a               f   
2     e         b           s         w       t    l               f   
3     p         x           y         w       t    p               f   
4     e         x           s         g       f    n               f   

  gill-spacing gill-size gill-color  ... stalk-surface-below-ring  \
0            c         n          k  ...                        s   
1            c         b          k  ...                        s   
2            c         b          n  ...                        s   
3            c         n          n  ...                        s   
4            w         b          k  ...                        s   

  stalk-color-above-ring stalk-color-below-ring veil-type veil-color  \
0                      w                      w         p          w   
1                      w                      w         p          w   
2                      w                      w         p          w   
3                      w                      w         p          w   
4                      w                      w         p          w   

  ring-number ring-type spore-print-color population habitat  
0           o         p                 k          s       u  
1           o         p                 n          n       g  
2           o         p                 n          n       m  
3           o         p                 k          s       u  
4           o         e                 n          a       g  

[5 rows x 23 columns]

In [7]:
shrooms_df_X = shrooms.drop(columns=['class'])
shrooms_df_y = shrooms['class']

In [8]:
shrooms_X = pd.get_dummies(data=shrooms_df_X)
shrooms_y = shrooms_df_y.replace({'e': 0, 'p': 1})

### Clean Cardio Dataset

### Clean Rain Dataset

### Clean BnB Dataset

### Clean Olympic Dataset

### Perform Trials

In [9]:
tree_params = [
    {
        'max_depth': [2,3,4,5,7,10,13,15,18,None], 
        'min_samples_split':[2,3,5,7,10,15,20],
        'min_samples_leaf':[2,3,5,7,10,15,20]
    }
]

log_reg_params = [
    {
        'penalty': ['l1', 'l2', 'elasticnet', 'none'],
        'C': 10 **np.array(np.arange(-8, 5, 1), dtype='float32')
    }
]

perceptron_params = [
    {
        'penalty': ['l1', 'l2', 'elasticnet', 'none'],
        'alpha': [0.00001, 0.0001, 0.001, 0.01, 0.1]
    }
]

svc_params = [
    {
        'kernel': ['linear'],
        'C': 10 **np.array(np.arange(-3, 2, 2), dtype='float32')
    },
    {
        'kernel': ['poly'],
        'degree': [2, 3],
        'C': 10 **np.array(np.arange(-3, 2, 2), dtype='float32'),
    },
    {
        'kernel': ['rbf'],
        'C': 10 **np.array(np.arange(-3, 2, 2), dtype='float32'),
        'gamma': [0.001,0.01,0.1,1,2]
    }
]

knn_params = [
    {
        'n_neighbors': np.arange(1, 106, 4),
        'metric': ["euclidean", "manhattan", "minkowski"]
    }
]

forest_params = [
    {
        'n_estimators': [1024],
        'min_samples_split': [1, 2, 4, 6, 8, 12, 16, 20]
    }
]

models = {
#     'tree': (DecisionTreeClassifier(), tree_params),
#     'log_reg': (LogisticRegression(), log_reg_params),
#     'perceptron': (Perceptron(), perceptron_params),
    'svm': (SVC(), svc_params),
#     'knn': (KNeighborsClassifier(), knn_params),
#     'forest': (RandomForestClassifier(), forest_params)
}

In [10]:
# perform trials on dataset
def perform_trials(dataset_name, data_X, data_y):
    results_columns = ['dataset', 'model', 'trial',
                       'train_accuracy', 'train_precision', 'train_recall', 'train_specificity',
                       'train_f1', 'train_auc', 'train_logloss',
                       'test_accuracy', 'test_precision', 'test_recall', 'test_specificity',
                       'test_f1', 'test_auc', 'test_logloss']
    num_trials = 1
    
    data_results = pd.DataFrame(columns=results_columns)

    # perform trials using each model
    for model_name in models.keys():
        
        model = models[model_name][0]
        model_params = models[model_name][1]
        
        train_metrics = np.zeros(7)
        test_metrics =  np.zeros(7)
        
        model_results = pd.DataFrame(columns=results_columns)
        
        # perform 5 trials on each dataset
        for trial_count in range(num_trials):
            # pick 5000 samples with replacement to be in the training set
            X_train, X_test, y_train, y_test = train_test_split(data_X, data_y, train_size=5000, random_state=trial_count)
            
            # grid search with 5 k-folds
            search = GridSearchCV(model, model_params, cv=5, verbose=3, n_jobs=-1)
            
            # find the best parameters for the model
            # grid search automatically refits a model on the entire validation set using the best parameters
            search.fit(X_train, y_train)
            
            # use metrics to evaluate model performance on the test set
            y_train_pred = search.predict(X_train)
            y_test_pred = search.predict(X_test)
            
            # compute metrics
            model_result = {
                'dataset': dataset_name,
                'model': model_name,
                'trial': trial_count + 1,

                'train_accuracy': accuracy_score(y_train, y_train_pred),
                'train_precision': precision_score(y_train, y_train_pred),
                'train_recall': recall_score(y_train, y_train_pred),
                'train_specificity': recall_score(y_train, y_train_pred, pos_label=0),
                'train_f1': f1_score(y_train, y_train_pred),
                'train_auc': roc_auc_score(y_train, y_train_pred),
                'train_logloss': log_loss(y_train, y_train_pred),

                'test_accuracy': accuracy_score(y_test, y_test_pred),
                'test_precision': precision_score(y_test, y_test_pred),
                'test_recall': recall_score(y_test, y_test_pred),
                'test_specificity': recall_score(y_test, y_test_pred, pos_label=0),
                'test_f1': f1_score(y_test, y_test_pred),
                'test_auc': roc_auc_score(y_test, y_test_pred),
                'test_logloss': log_loss(y_test, y_test_pred)
            }
            
            # append model_result to the model_results dataframe
            model_results = model_results.append(model_result, ignore_index=True)
        
        # append model_results to data_results
        data_results = data_results.append(model_results, ignore_index=True)
        
        avg_result = {
            'dataset': dataset_name,
            'model': model_name,
            'trial': 'avg',
            
            'train_accuracy': model_results.train_accuracy.mean(),
            'train_precision': model_results.train_precision.mean(),
            'train_recall': model_results.train_recall.mean(),
            'train_specificity': model_results.train_specificity.mean(),
            'train_f1': model_results.train_f1.mean(),
            'train_auc': model_results.train_auc.mean(),
            'train_logloss': model_results.train_logloss.mean(),
            
            'test_accuracy': model_results.test_accuracy.mean(),
            'test_precision': model_results.test_precision.mean(),
            'test_recall': model_results.test_recall.mean(),
            'test_specificity': model_results.test_specificity.mean(),
            'test_f1': model_results.test_f1.mean(),
            'test_auc': model_results.test_auc.mean(),
            'test_logloss': model_results.test_logloss.mean()
        }
        
        # append avg_result to the data_results dataframe
        data_results = data_results.append(avg_result, ignore_index=True)
    
    return data_results

### Duy Results

In [11]:
chess_results = perform_trials('chess', chess_X, chess_y)
chess_results.to_csv('results/chess')
chess_results

Fitting 5 folds for each of 90 candidates, totalling 450 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  16 tasks      | elapsed:  1.5min
[Parallel(n_jobs=-1)]: Done  17 tasks      | elapsed:  1.5min
[Parallel(n_jobs=-1)]: Done  18 tasks      | elapsed:  1.5min
[Parallel(n_jobs=-1)]: Done  19 tasks      | elapsed:  1.5min
[Parallel(n_jobs=-1)]: Done  20 tasks      | elapsed:  1.5min
[Parallel(n_jobs=-1)]: Done  21 tasks      | elapsed:  1.5min
[Parallel(n_jobs=-1)]: Done  22 tasks      | elapsed:  1.5min
[Parallel(n_jobs=-1)]: Done  23 tasks      | elapsed:  1.5min
[Parallel(n_jobs=-1)]: Done  24 tasks      | elapsed:  1.5min
[Parallel(n_jobs=-1)]: Done  25 tasks      | elapsed:  1.5min
[Parallel(n_jobs=-1)]: Done  26 tasks      | elapsed:  1.5min
[Parallel(n_jobs=-1)]: Done  27 tasks      | elapsed:  1.5min
[Parallel(n_jobs=-1)]: Done  28 tasks      | elapsed:  1.5min
[Parallel(n_jobs=-1)]: Done  29 tasks      | elapsed:  1.5min
[Parallel(n_jobs=-1)]: Done  30 tasks      | elapsed:  1

KeyboardInterrupt: 

In [ ]:
shrooms_results = perform_trials('shrooms', shrooms_X, shrooms_y)
shrooms_results.to_csv('results/shrooms')
shrooms_results

In [ ]:
# combine datasets

In [ ]:
# create table 1

In [ ]:
# create table 2

In [ ]:
# create table 3